# **Load Dataset**

Membuat direktori .kaggle untuk menyimpan file API key dari Kaggle.

In [1]:
!mkdir -p ~/.kaggle

Mengunggah file API key dari lokal ke Colab.

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"irgisetiawan","key":"bc41477d4b375913aa3fc1ee0c2e697f"}'}

Mengunduh dataset Bookcrossing dari Kaggle menggunakan API key yang telah diunggah

In [3]:
!kaggle datasets download -d ruchi798/bookcrossing-dataset

Dataset URL: https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset
License(s): CC0-1.0
 99% 75.0M/76.1M [00:01<00:00, 65.2MB/s]
100% 76.1M/76.1M [00:01<00:00, 65.1MB/s]


Mengekstrak file dataset Bookcrossing yang telah diunduh dari file ZIP.

In [4]:
!unzip bookcrossing-dataset.zip

Archive:  bookcrossing-dataset.zip
  inflating: Book reviews/Book reviews/BX-Book-Ratings.csv  
  inflating: Book reviews/Book reviews/BX-Users.csv  
  inflating: Book reviews/Book reviews/BX_Books.csv  
  inflating: Books Data with Category Language and Summary/Preprocessed_data.csv  


Memasang library annoy untuk pencarian approximate nearest neighbors dan fuzzywuzzy untuk pencocokan teks berbasis string.

In [5]:
!pip install annoy
!pip install fuzzywuzzy[speedup]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552446 sha256=9e2b8e71c7c4c89f39f86e0e904fe5c7f4b5a0f9e9ba509b465beff1a0b71a40
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 17.7 MB/s eta 0:00:00


# **Preprocessing Data**

Mengimpor library yang dibutuhkan untuk pemrosesan data, ekstraksi fitur teks dengan TF-IDF, pencarian nearest neighbors, pencocokan teks menggunakan fuzzy matching, pembagian dataset, evaluasi metrik, dan pengurangan dimensi menggunakan TruncatedSVD.

In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.decomposition import TruncatedSVD

Membaca file CSV yang berisi data buku, penilaian, dan pengguna dari folder 'Book reviews' dengan memisahkan kolom menggunakan delimiter ;, menangani kesalahan baris dengan parameter on_bad_lines='skip', dan menggunakan encoding latin-1 untuk mengatasi karakter khusus.

In [7]:
books = pd.read_csv('/content/Book reviews/Book reviews/BX_Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
ratings = pd.read_csv('/content/Book reviews/Book reviews/BX-Book-Ratings.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
users = pd.read_csv('/content/Book reviews/Book reviews/BX-Users.csv', sep=';', encoding='latin-1', on_bad_lines='skip')

Menampilkan lima baris pertama dari masing-masing dataset: buku, penilaian, dan pengguna, untuk memeriksa isi dan struktur data.

In [8]:
print("Books Dataset:")
print(books.head())

print("\nRatings Dataset:")
print(ratings.head())

print("\nUsers Dataset:")
print(users.head())

Books Dataset:
         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author  Year-Of-Publication                Publisher  \
0    Mark P. O. Morford                 2002  Oxford University Press   
1  Richard Bruce Wright                 2001    HarperFlamingo Canada   
2          Carlo D'Este                 1991          HarperPerennial   
3      Gina Bari Kolata                 1999     Farrar Straus Giroux   
4       E. J. W. Barber                 1999   W. W. Norton & Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.c

Menampilkan informasi detail tentang dataset buku, penilaian, dan pengguna, termasuk jumlah entri, tipe data di setiap kolom, dan apakah ada nilai yang hilang.

In [9]:
print("\nBooks Dataset Info:")
print(books.info())

print("\nRatings Dataset Info:")
print(ratings.info())

print("\nUsers Dataset Info:")
print(users.info())


Books Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271379 non-null  object
 1   Book-Title           271379 non-null  object
 2   Book-Author          271377 non-null  object
 3   Year-Of-Publication  271379 non-null  int64 
 4   Publisher            271377 non-null  object
 5   Image-URL-S          271379 non-null  object
 6   Image-URL-M          271379 non-null  object
 7   Image-URL-L          271379 non-null  object
dtypes: int64(1), object(7)
memory usage: 16.6+ MB
None

Ratings Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   B

Memeriksa jumlah nilai yang hilang pada setiap kolom dataset buku, menghitung jumlah ISBN unik (buku unik), dan menampilkan statistik deskriptif untuk kolom 'Year-Of-Publication'.

In [10]:
print("Books Dataset - Checking for missing values:")
print(books.isnull().sum())

print("\nUnique Books:")
print(books['ISBN'].nunique())

print("\nBooks Dataset - Descriptive statistics for Year-Of-Publication:")
print(books['Year-Of-Publication'].describe())

Books Dataset - Checking for missing values:
ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
dtype: int64

Unique Books:
271379

Books Dataset - Descriptive statistics for Year-Of-Publication:
count    271379.000000
mean       1959.756050
std         258.011363
min           0.000000
25%        1989.000000
50%        1995.000000
75%        2000.000000
max        2050.000000
Name: Year-Of-Publication, dtype: float64


Memeriksa nilai yang hilang dalam dataset penilaian, serta menghitung jumlah pengguna dan buku unik yang terdapat dalam dataset tersebut.

In [11]:
print("Ratings Dataset - Checking for missing values:")
print(ratings.isnull().sum())

print("\nUnique Users and Books in Ratings:")
print(f"Unique Users: {ratings['User-ID'].nunique()}")
print(f"Unique Books: {ratings['ISBN'].nunique()}")

Ratings Dataset - Checking for missing values:
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

Unique Users and Books in Ratings:
Unique Users: 105283
Unique Books: 340556


Memeriksa jumlah nilai yang hilang pada dataset pengguna dan menampilkan statistik deskriptif untuk kolom 'Age'.

In [12]:
print("\nUsers Dataset - Checking for missing values:")
print(users.isnull().sum())

print("\nUsers Dataset - Descriptive statistics for Age:")
print(users['Age'].describe())


Users Dataset - Checking for missing values:
User-ID          0
Location         0
Age         110762
dtype: int64

Users Dataset - Descriptive statistics for Age:
count    168096.000000
mean         34.751434
std          14.428097
min           0.000000
25%          24.000000
50%          32.000000
75%          44.000000
max         244.000000
Name: Age, dtype: float64


Membersihkan dataset dengan cara menghapus baris yang memiliki nilai kosong pada kolom penting (ISBN, judul buku, pengarang, tahun terbit, dan penerbit), menghapus duplikat berdasarkan ISBN, serta memfilter pengguna dengan usia antara 5 dan 100 tahun.

In [13]:
books_clean = books.dropna(subset=['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher'])
ratings_clean = ratings.dropna()
users_clean = users.dropna(subset=['User-ID', 'Age'])

books_clean = books_clean.drop_duplicates(subset='ISBN')

users_clean = users_clean[(users_clean['Age'] >= 5) & (users_clean['Age'] <= 100)]

# **Feature Engineering**

Membuat fitur gabungan pada dataset buku dengan menggabungkan judul buku, pengarang, dan penerbit ke dalam satu kolom yang disebut 'combined_features', lalu menampilkan lima baris pertama dari kolom tersebut.

In [14]:
books_clean['combined_features'] = books_clean['Book-Title'] + ' ' + books_clean['Book-Author'] + ' ' + books_clean['Publisher']

print(books_clean['combined_features'].head())

0    Classical Mythology Mark P. O. Morford Oxford ...
1    Clara Callan Richard Bruce Wright HarperFlamin...
2    Decision in Normandy Carlo D'Este HarperPerennial
3    Flu: The Story of the Great Influenza Pandemic...
4    The Mummies of Urumchi E. J. W. Barber W. W. N...
Name: combined_features, dtype: object


Menggunakan TF-IDF Vectorizer untuk mengubah fitur gabungan ('combined_features') menjadi representasi numerik, menghilangkan kata-kata umum (stop words) dalam bahasa Inggris, dan menampilkan ukuran matriks TF-IDF yang dihasilkan.

In [15]:
tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(books_clean['combined_features'])

print(f"Ukuran TF-IDF Matrix: {tfidf_matrix.shape}")

Ukuran TF-IDF Matrix: (271375, 116788)


Membangun model Nearest Neighbors menggunakan metrik cosine similarity dan algoritma brute-force untuk menemukan buku-buku yang mirip berdasarkan fitur TF-IDF yang telah diekstraksi.

In [16]:
model_nn = NearestNeighbors(metric='cosine', algorithm='brute')
model_nn.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

Mendefinisikan fungsi untuk mendapatkan rekomendasi buku menggunakan model Nearest Neighbors. Fungsi ini mencari buku yang mirip dengan buku yang diberikan berdasarkan fitur TF-IDF. Jika buku tidak ditemukan, pesan kesalahan ditampilkan. Menampilkan hasil rekomendasi untuk buku 'Harry Potter and the Sorcerer's Stone'.

In [17]:
indices = pd.Series(books_clean.index, index=books_clean['Book-Title']).drop_duplicates()

def get_nn_recommendations(title, n_recommendations=10):
    if title not in indices:
        return "Buku tidak ditemukan."

    idx = indices[title]

    distances, indices_nn = model_nn.kneighbors(tfidf_matrix[idx], n_neighbors=n_recommendations + 1)

    recommended_indices = indices_nn[0][1:]

    return books_clean['Book-Title'].iloc[recommended_indices]

print(get_nn_recommendations('Harry Potter and the Sorcerer\'s Stone'))

52724                     An American Killing
129304                Love Her Madly: A Novel
128062    She's Not There: A Poppy Rice Novel
264643                     The Book of Phoebe
66324                         Brown-Eyed Girl
140398                       Gabriel's Lament
208207                       Gabriel's Lament
204078                      Way to Go, Smith!
83117                       Mary Ann and Bill
221632                    Lament a Lost Lover
Name: Book-Title, dtype: object


Menghapus nilai kosong dalam dataset penilaian, lalu menggabungkan dataset penilaian yang bersih dengan dataset buku yang bersih berdasarkan kolom ISBN. Menampilkan lima baris pertama dari hasil penggabungan.

In [18]:
ratings_clean = ratings.dropna()
ratings_books = pd.merge(ratings_clean, books_clean, on='ISBN')

print(ratings_books.head())

   User-ID        ISBN  Book-Rating            Book-Title Book-Author  \
0   276725  034545104X            0  Flesh Tones: A Novel  M. J. Rose   
1     2313  034545104X            5  Flesh Tones: A Novel  M. J. Rose   
2     6543  034545104X            0  Flesh Tones: A Novel  M. J. Rose   
3     8680  034545104X            5  Flesh Tones: A Novel  M. J. Rose   
4    10314  034545104X            9  Flesh Tones: A Novel  M. J. Rose   

   Year-Of-Publication         Publisher  \
0                 2002  Ballantine Books   
1                 2002  Ballantine Books   
2                 2002  Ballantine Books   
3                 2002  Ballantine Books   
4                 2002  Ballantine Books   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/034545104X.0...   
1  http://images.amazon.com/images/P/034545104X.0...   
2  http://images.amazon.com/images/P/034545104X.0...   
3  http://images.amazon.com/images/P/034545104X.0...   
4  http://images

# **Building Recommendation System**

Mendefinisikan fungsi fuzzy_match yang menggunakan fuzzy string matching untuk menemukan judul buku yang paling mirip dengan judul yang diberikan. Fungsi ini akan mengembalikan sejumlah judul yang paling cocok berdasarkan batas yang ditentukan.

In [19]:
def fuzzy_match(title, books, limit=3):
    matches = process.extract(title, books, limit=limit)
    return matches

Mendefinisikan fungsi get_fuzzy_nn_recommendations yang pertama kali mencari judul buku terdekat menggunakan fuzzy matching. Setelah judul yang paling cocok ditemukan, fungsi ini menggunakan model Nearest Neighbors untuk memberikan rekomendasi buku berdasarkan judul tersebut. Jika tidak ada judul yang cocok ditemukan, fungsi akan menampilkan pesan kesalahan.

In [20]:
def get_fuzzy_nn_recommendations(title, n_recommendations=10):
    matches = fuzzy_match(title, books_clean['Book-Title'].values)

    if not matches:
        return "Tidak ada judul yang cocok ditemukan."

    matched_title = matches[0][0]

    print(f"Menampilkan rekomendasi untuk: '{matched_title}'")

    idx = indices[matched_title]

    distances, indices_nn = model_nn.kneighbors(tfidf_matrix[idx], n_neighbors=n_recommendations + 1)

    recommended_indices = indices_nn[0][1:]

    return books_clean['Book-Title'].iloc[recommended_indices]

Mendefinisikan fungsi recommend_for_user yang memberikan rekomendasi buku untuk pengguna berdasarkan buku yang telah diberi rating tertinggi oleh pengguna tersebut. Fungsi ini mengambil beberapa buku yang diberi rating tertinggi oleh pengguna, menggunakan model Nearest Neighbors untuk memberikan rekomendasi, lalu menghilangkan duplikasi dari daftar rekomendasi akhir.

In [21]:
def recommend_for_user(user_id, ratings_books, n_recommendations=10):
    user_ratings = ratings_books[ratings_books['User-ID'] == user_id]

    if user_ratings.empty:
        return "Pengguna tidak ditemukan atau belum memberikan rating."

    top_rated_books = user_ratings.sort_values(by='Book-Rating', ascending=False).head(5)

    recommended_books = []
    for book in top_rated_books['Book-Title'].values:
        recommended_books.extend(get_nn_recommendations(book, n_recommendations))

    recommended_books = list(dict.fromkeys(recommended_books))

    return recommended_books[:n_recommendations]

Mendefinisikan fungsi interactive_recommendation untuk menjalankan rekomendasi secara interaktif. Jika input berupa ID pengguna (digit), rekomendasi diberikan berdasarkan preferensi pengguna tersebut. Jika input berupa judul buku (teks), rekomendasi diberikan berdasarkan judul buku dengan bantuan fuzzy matching. Fungsi ini dijalankan dua kali untuk menguji interaktivitas.

In [22]:
def interactive_recommendation():
    user_input = input("Masukkan judul buku atau ID pengguna: ")

    if user_input.isdigit():
        user_id = int(user_input)
        print(f"Rekomendasi untuk pengguna {user_id}:")
        print(recommend_for_user(user_id, ratings_books))
    else:
        print(f"Rekomendasi untuk buku '{user_input}':")
        print(get_fuzzy_nn_recommendations(user_input))

interactive_recommendation()
interactive_recommendation()

Masukkan judul buku atau ID pengguna: The Joy Luck Club
Rekomendasi untuk buku 'The Joy Luck Club':
Menampilkan rekomendasi untuk: 'The Joy Luck Club'
42378                              The Joy Luck Club
41356                                  Joy Luck Club
32773                              The Joy Luck Club
211474                            Joy Luck Club-O.M.
13759     The Joy Luck Club (Vintage Contemporaries)
207429                               Beginner's Luck
151326                                 The Moon Lady
82694                      The Hundred Secret Senses
10715                      The Hundred Secret Senses
54623                      The Hundred Secret Senses
Name: Book-Title, dtype: object
Masukkan judul buku atau ID pengguna: 276798
Rekomendasi untuk pengguna 276798:
['Ein Mord fÃ?Â¼r Kay Scarpetta', 'Ein Fall fÃ?Â¼r Kay Scarpetta.', 'Brandherd 5 CDs. Ein Fall fÃ?Â¼r Kay Scarpetta.', 'Blinder Passagier. Ein neuer Fall fÃ?Â¼r Kay Scarpetta.', 'Blinder Passagier. Ein Kay- 

# **Evaluation**

Membagi dataset penilaian menjadi data latih (80%) dan data uji (20%) menggunakan train_test_split. Menampilkan ukuran data latih dan data uji untuk memverifikasi pembagian.

In [23]:
train_data, test_data = train_test_split(ratings_clean, test_size=0.2, random_state=42)

print(f"Train data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")

Train data size: (919824, 3)
Test data size: (229956, 3)


Mendefinisikan fungsi evaluate_recommendations untuk mengevaluasi rekomendasi yang diberikan kepada seorang pengguna berdasarkan precision, recall, dan F1-score. Fungsi ini memeriksa buku yang diberi rating tinggi (>= 8) oleh pengguna sebagai 'true positives', lalu membandingkannya dengan buku yang direkomendasikan. Metrik precision, recall, dan F1 dihitung untuk menilai akurasi rekomendasi.

In [36]:
def evaluate_recommendations(user_id, model_nn, n_recommendations=10):
    user_test_ratings = test_data[test_data['User-ID'] == user_id]

    if user_test_ratings.empty:
        return None

    true_positive_books = user_test_ratings[user_test_ratings['Book-Rating'] >= 8]['ISBN'].values

    recommended_books = recommend_for_user(user_id, ratings_books, n_recommendations)

    if not recommended_books:
        return None

    if isinstance(recommended_books, str):
        recommended_books = [recommended_books]

    recommended_books_isbn = books_clean[books_clean['Book-Title'].isin(recommended_books)]['ISBN'].values

    y_true = [1 if isbn in true_positive_books else 0 for isbn in recommended_books_isbn]
    y_pred = [1] * len(recommended_books_isbn)

    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred, zero_division=1)
    f1 = f1_score(y_true, y_pred, zero_division=1)

    return precision, recall, f1

Mendefinisikan fungsi evaluate_for_sampled_users_in_batch yang mengevaluasi rekomendasi untuk sekelompok pengguna secara batch. Fungsi ini menghitung precision, recall, dan F1-score rata-rata untuk pengguna yang diambil secara acak dari data uji. Evaluasi dilakukan secara batch untuk efisiensi. Hasil akhirnya adalah precision, recall, dan F1-score rata-rata untuk semua pengguna yang diuji.

In [39]:
sampled_user_ids = np.random.choice(test_data['User-ID'].unique(), size=int(0.1 * len(test_data['User-ID'].unique())), replace=False)

def evaluate_for_sampled_users_in_batch(test_data, model_nn, sampled_user_ids, batch_size=100, n_recommendations=10):
    precisions = []
    recalls = []
    f1_scores = []

    for i in range(0, len(sampled_user_ids), batch_size):
        batch_user_ids = sampled_user_ids[i:i + batch_size]
        for user_id in batch_user_ids:
            result = evaluate_recommendations(user_id, model_nn, n_recommendations)
            if result:
                precision, recall, f1 = result
                precisions.append(precision)
                recalls.append(recall)
                f1_scores.append(f1)

    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0
    avg_f1 = sum(f1_scores) / len(f1_scores) if f1_scores else 0

    return avg_precision, avg_recall, avg_f1

avg_precision, avg_recall, avg_f1 = evaluate_for_sampled_users_in_batch(test_data, model_nn, sampled_user_ids, batch_size=100)

print(f"Rata-rata Precision: {avg_precision}")
print(f"Rata-rata Recall: {avg_recall}")
print(f"Rata-rata F1-score: {avg_f1}")

Rata-rata Precision: 0.08172273640849355
Rata-rata Recall: 1.0
Rata-rata F1-score: 0.08844182146442586
